### 1. Load data

In [1]:
import torch
import os
from torch.utils import data

In [2]:
data_folder = 'data/'

In [3]:
class EuroSAT(data.Dataset):
    def __init__(self,dataset,transform = None):
        self.dataset = dataset
        self.transform = transform

    def __getitem__(self,index):
        if self.transform:
            x = self.transform(dataset[index][0])
        else:
            x = dataset[index][0]
        y = dataset[index][1]
        return x,y
    
    def __len__(self):
        return len(dataset)


In [4]:
train_loader = torch.load(os.path.join(data_folder,'train_loader.pth'))
val_loader = torch.load(os.path.join(data_folder,'val_loader.pth'))
test_loader = torch.load(os.path.join(data_folder,'test_loader.pth'))

<ipython-input-4-0b32d987e57a>:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  train_loader = torch.load(os.path.join(data_folder,'train_loader.pth'))
/home/nirzaree/anacon

In [5]:
batch_size = 16 #from previous notebook

In [6]:
print(len(train_loader.dataset))
print(len(val_loader.dataset))
print(len(test_loader.dataset))

18900
4050
4050


In [7]:
(len(train_loader.dataset)) + (len(val_loader.dataset)) + (len(test_loader.dataset))

27000

The sizes match the original dataset size and the splits sizes

### 2. Load model

In [8]:
import torchvision
from torchvision import models

Remember to upgrade torch and torchvision in case the models method doesn't support the weights (I had a really old installation and had to upgrade)

In [9]:
model = models.resnet50(weights=models.ResNet50_Weights.DEFAULT)


In [10]:
import torchsummary

In [11]:
# Check is GPU is enabled
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("Device: {}".format(device))

# Get specific GPU model
if str(device) == "cuda:0":
  print("GPU: {}".format(torch.cuda.get_device_name(0)))

Device: cpu


In [12]:
model.fc

Linear(in_features=2048, out_features=1000, bias=True)

In [13]:
NUM_CLASSES = 10

In [14]:
model.fc = torch.nn.Linear(model.fc.in_features,NUM_CLASSES)

In [15]:
model = model.to(device)

In [33]:
torchsummary.summary(model,input_size=(3,224,224))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 112, 112]           9,408
       BatchNorm2d-2         [-1, 64, 112, 112]             128
              ReLU-3         [-1, 64, 112, 112]               0
         MaxPool2d-4           [-1, 64, 56, 56]               0
            Conv2d-5           [-1, 64, 56, 56]           4,096
       BatchNorm2d-6           [-1, 64, 56, 56]             128
              ReLU-7           [-1, 64, 56, 56]               0
            Conv2d-8           [-1, 64, 56, 56]          36,864
       BatchNorm2d-9           [-1, 64, 56, 56]             128
             ReLU-10           [-1, 64, 56, 56]               0
           Conv2d-11          [-1, 256, 56, 56]          16,384
      BatchNorm2d-12          [-1, 256, 56, 56]             512
           Conv2d-13          [-1, 256, 56, 56]          16,384
      BatchNorm2d-14          [-1, 256,

### Specify model training parameters

In [17]:
n_epochs = 10
lr = 0.001


In [18]:
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(),lr=lr)

What torch.max() does

In [35]:
a = torch.randn(4,4)
a

tensor([[ 0.0519,  0.6531,  0.6558,  0.0101],
        [-0.9179, -0.3228, -0.7401, -1.6503],
        [-0.4486, -0.4789, -0.1908, -0.4035],
        [-1.7116,  0.6079,  0.0411,  0.6539]])

In [37]:
torch.max(a,1)

torch.return_types.max(
values=tensor([ 0.6558, -0.3228, -0.1908,  0.6539]),
indices=tensor([2, 1, 2, 3]))

In [19]:
import tqdm

In [24]:
def train(model,dataloader, criterion, optimizer):
    model.train()

    running_loss = 0.0
    running_total_correct = 0.0

    for i,(inputs,labels) in enumerate(tqdm(dataloader)):
        inputs = inputs.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()

        outputs = model(inputs)

        loss = criterion(outputs,labels)

        loss.backward()

        optimizer.step()

        _,preds = torch.max(outputs,1)

        running_loss += loss.item() *inputs.size(0)
        running_total_correct += torch.sum(preds == labels) 

    epoch_loss = running_loss / len(dataloader.dataset)
    epoch_accuracy = (running_total_correct /  len(dataloader.dataset)) * 100
    print(f'Train loss : {epoch_loss:.2f};Accuracy : {epoch_accuracy:.2f}')

    return epoch_loss, epoch_accuracy

### Model evaluation function

In [23]:
def evaluate(model,dataloader,criterion,phase='val'):
    model.eval()

    running_loss = 0.0
    running_total_correct = 0.0

    for i,(inputs,labels) in enumerate(tqdm(dataloader)):
        inputs = inputs.to(device)
        labels = labels.to(device)

        with torch.set_grad_enabled(False):
            outputs = model(inputs)
            loss = criterion(outputs,labels)
            _, preds = torch.max(outputs,1)
        
        running_loss += loss.item() * inputs.size(0)
        running_total_correct += torch.sum(preds == labels) 
    
    epoch_loss = running_loss / len(dataloader.dataset)
    epoch_accuracy = (running_total_correct / len(dataloader.dataset)) * 100
    print(f'{phase.title()} Loss = {epoch_loss:.2f}; Accuracy = {epoch_accuracy:.2f}')

    return epoch_loss,epoch_accuracy

### Model fit function

In [22]:
import numpy as np

In [25]:
def fit(model,train_loader,val_loader,n_epochs,lr,criterion,optimizer):
    best_loss = np.inf
    best_model = None

    for epoch in range(n_epochs):
        print(f'Epoch : {epoch+1}')
        train(model,train_loader,criterion,optimizer)
        val_loss, = evaluate(model,val_loader,criterion)

        if val_loss < best_loss:
            best_loss = val_loss
            best_model = model
            
    return best_model

In [30]:
enumerate(train_loader)

AttributeError: 'DataLoader' object has no attribute 'pin_memory_device'

In [31]:
torch.__version__

'2.4.1+cu121'

In [27]:
train(model,train_loader,criterion,optimizer)

TypeError: 'module' object is not callable

In [26]:
best_model = fit(model,train_loader,val_loader,n_epochs,lr,criterion,optimizer)

Epoch : 1


TypeError: 'module' object is not callable